In [0]:
df = spark.read.format("csv").\
    option("header", True).\
        option("inferSchema",True).\
            load("/Volumes/pyspark_dbt_project/source/source_data/Customers/")
display(df)

In [0]:
schema_customer= df.schema
schema_customer

### **SPARK STREAMING**

In [0]:
entities = ["customers", "trips", "vehicles","drivers", "payments", "locations"]

#dynamic solution 

In [0]:
for entity in entities:
    df_batch = spark.read.format("csv").\
        option("header", True).\
        option("inferSchema",True).\
        load(f"/Volumes/pyspark_dbt_project/source/source_data/{entity}/")
    
    schema_entity = df_batch.schema


    df=spark.readStream.format("csv").\
        option("header", True).\
        schema(schema_entity).\
        load(f"/Volumes/pyspark_dbt_project/source/source_data/{entity}/")


    df.writeStream.format("delta").\
        outputMode("append").\
        option("checkpointLocation",f"/Volumes/pyspark_dbt_project/bronze/checkpoint/{entity}").\
        trigger(once=True).\
        toTable(f"pyspark_dbt_project.bronze.{entity}")
